## Dependencies

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

pd.set_option('max_columns', 50)

pd.read_csv('hotel_bookings_data.csv')## Load the dataset

In [2]:
df = pd.read_csv('hotel_bookings_data.csv')
df.sample(3)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_weekdays_nights,adults,children,babies,meal,city,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status
3281,Resort Hotel,0,31,2017,February,49,3,0,1,2,0.0,0,Breakfast,Kota Jambi,Online TA,TA/TO,0,0,0,0,No Deposit,240.0,NaN,0,Personal,61.2,0,1,Check-Out
55111,City Hotel,1,159,2018,October,32,5,0,1,1,1.0,0,Breakfast,Kota Denpasar,Online TA,TA/TO,0,0,0,0,No Deposit,8.0,NaN,0,Personal,105.3,0,0,Canceled
2995,Resort Hotel,1,42,2017,January,46,10,0,5,2,0.0,0,Breakfast,Kota Denpasar,Direct,Direct,0,0,0,0,No Deposit,250.0,NaN,0,Personal,47.7,0,0,Canceled


In [3]:
categorical = [
    'hotel', 'is_canceled', 'meal', 'city', 'market_segment',
    'distribution_channel', 'is_repeated_guest', 'reservation_status',
    'deposit_type', 'agent', 'company', 'customer_type'
]
numerical = [
    'adults', 'children', 'babies', 'previous_cancellations',
    'previous_bookings_not_canceled', 'booking_changes', 'adr',
    'required_car_parking_spaces', 'total_of_special_requests'	
    
]
time_related = [
    'lead_time', 'arrival_date_year', 'arrival_date_month',
    'arrival_date_week_number', 'arrival_date_day_of_month', 
    'stays_in_weekend_nights', 'stays_in_weekdays_nights', 
    'days_in_waiting_list',
]

## Pre-exploration

### Summary of dataset

In [51]:
from common_function import dataset_summary
summary = dataset_summary(df)
summary

,variable,no_unique,pandas_dtype,missing_value,%_missing_values,unique_value
0,hotel,2,object,0,0.0,"[Resort Hotel, City Hotel]"
1,is_canceled,2,int64,0,0.0,"[0, 1]"
2,lead_time,479,int64,0,0.0,"[342, 737, 7, 13, 14, 0, 9, 85, 75, 23, 35, 68..."
3,arrival_date_year,3,int64,0,0.0,"[2017, 2018, 2019]"
4,arrival_date_month,12,object,0,0.0,"[September, October, November, December, Janua..."
5,arrival_date_week_number,53,int64,0,0.0,"[27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 3..."
6,arrival_date_day_of_month,31,int64,0,0.0,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
7,stays_in_weekend_nights,17,int64,0,0.0,"[0, 1, 2, 4, 3, 6, 13, 8, 5, 7, 12, 9, 16, 18,..."
8,stays_in_weekdays_nights,33,int64,0,0.0,"[0, 1, 2, 3, 4, 5, 10, 11, 8, 6, 7, 15, 9, 12,..."
9,adults,14,int64,0,0.0,"[2, 1, 3, 4, 40, 26, 50, 27, 55, 20, 6, 5, 10, 0]"


**Observations**
* Most of variables have a correct pandas dtype except children, agent, company and adr.
* There are four variables that have missing values such as children, city, agent and company. However, one of them, company, has a missing values more than 90% of total rows.
* Some of variables contain an unreliable value. Both stays_in_weekend_nights and stays_in_weekdays_nights have a unique value 0 and also adults, children and babies have it too. All these variables need a further investigation about their unique values.
* Agent and company are an unique identifier so the missing values on them could be interpreted as a guest did not book the hotel via an agent or a company.

### Data quality issues

In [5]:
df['stays_in_weekend_nights'].value_counts()

0     51998
2     33308
1     30626
4      1855
3      1259
6       153
5        79
8        60
7        19
9        11
10        7
12        5
13        3
16        3
14        2
18        1
19        1
Name: stays_in_weekend_nights, dtype: int64

In [6]:
df['stays_in_weekdays_nights'].value_counts()

2     33684
1     30310
3     22258
5     11077
4      9563
0      7645
6      1499
10     1036
7      1029
8       656
9       231
15       85
11       56
19       44
12       42
20       41
14       35
13       27
16       16
21       15
22        7
25        6
18        6
30        5
17        4
24        3
40        2
33        1
42        1
50        1
32        1
26        1
34        1
35        1
41        1
Name: stays_in_weekdays_nights, dtype: int64

Both variables, stay in weekdays and weekends, probably could have a value of 0 so they need to be investigated further.

In [7]:
df['adults'].value_counts()

2     89680
1     23027
3      6202
0       403
4        62
26        5
27        2
20        2
5         2
40        1
50        1
55        1
6         1
10        1
Name: adults, dtype: int64

Typically, the room hotel capacity is only 1-4 guests but in this dataset there are some unusual values which are 0 and more than 4 guests.

In [8]:
df['children'].value_counts()

0.0     110796
1.0       4861
2.0       3652
3.0         76
10.0         1
Name: children, dtype: int64

In [9]:
df['babies'].value_counts()

0     118473
1        900
2         15
10         1
9          1
Name: babies, dtype: int64

Most of guests did not bring their children or babies to the hotel as seen in value counts above. However, notice that some guests have 10 children or 9-10 babies. Let's take a look this strange values.

In [10]:
babies_filter = (df['children'] > 3) | (df['babies'] > 2)
df[babies_filter]

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_weekdays_nights,adults,children,babies,meal,city,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status
328,Resort Hotel,1,55,2017,September,29,12,4,10,2,10.0,0,Breakfast,Kota Denpasar,Offline TA/TO,TA/TO,0,0,0,2,No Deposit,8.0,NaN,0,Contract,133.16,0,1,No-Show
46619,City Hotel,0,37,2018,March,3,12,0,2,2,0.0,10,Breakfast,Kota Denpasar,Online TA,TA/TO,0,0,0,1,No Deposit,9.0,NaN,0,Personal,84.45,0,1,Check-Out
78656,City Hotel,0,11,2017,December,42,11,2,1,1,0.0,9,Breakfast,Kabupaten Bangka,Corporate,Corporate,0,0,0,1,No Deposit,95.0,NaN,0,Family,95.00,0,0,Check-Out


Since the adults are not zero, it can be assumed that there is no issue with 10 children or 10 babies.

In [11]:
df['meal'].value_counts()

Breakfast     92310
Dinner        14463
No Meal       10650
Undefined      1169
Full Board      798
Name: meal, dtype: int64

The 'undefined' must be replaced as 'No meal' since undefined means no meal is provided.

Let's investigate the data quality issues.

In [12]:
zero_guest = (
    (df['adults'] == 0) & 
    (df['children'] == 0) & 
    (df['babies'] == 0)
)
df[zero_guest].sample(5)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_weekdays_nights,adults,children,babies,meal,city,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status
94769,City Hotel,0,4,2018,October,32,5,2,2,0,0.0,0,No Meal,Kota Jakarta Selatan,Online TA,TA/TO,0,0,0,2,No Deposit,9.0,NaN,0,Personal,69.50,0,2,Check-Out
42591,City Hotel,0,0,2017,November,37,7,1,4,0,0.0,0,Breakfast,Kota Surabaya,Online TA,TA/TO,0,0,0,2,No Deposit,11.0,NaN,0,Personal,0.00,0,1,Check-Out
41020,City Hotel,0,0,2017,October,33,10,9,20,0,0.0,0,No Meal,Kota Denpasar,Online TA,Direct,0,0,0,20,No Deposit,NaN,47.0,0,Personal,0.00,0,0,Check-Out
108234,City Hotel,0,0,2019,May,12,24,0,0,0,0.0,0,Breakfast,Kota Denpasar,Online TA,TA/TO,1,0,0,0,No Deposit,86.0,NaN,0,Personal,0.00,0,0,Check-Out
104516,City Hotel,0,125,2019,March,1,1,4,10,0,0.0,0,No Meal,Kota Denpasar,Online TA,TA/TO,0,0,0,2,No Deposit,9.0,NaN,0,Personal,27.69,0,2,Check-Out


In [13]:
zero_night = (
    (df['stays_in_weekend_nights'] == 0) & 
    (df['stays_in_weekdays_nights'] == 0)
)
df[zero_night].sample(5)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_weekdays_nights,adults,children,babies,meal,city,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status
25366,Resort Hotel,0,1,2018,August,26,24,0,0,1,0.0,0,Breakfast,Kota Denpasar,Online TA,TA/TO,0,0,0,0,No Deposit,240.0,NaN,0,Personal,0.0,0,0,Check-Out
80782,City Hotel,0,0,2017,January,48,23,0,0,2,0.0,0,Breakfast,Kota Denpasar,Online TA,TA/TO,1,0,0,0,No Deposit,8.0,NaN,0,Personal,0.0,0,0,Check-Out
100384,City Hotel,0,0,2018,December,44,28,0,0,2,0.0,0,No Meal,Kota Denpasar,Online TA,TA/TO,1,0,0,0,No Deposit,8.0,NaN,0,Personal,0.0,0,1,Check-Out
20874,Resort Hotel,0,2,2018,April,7,13,0,0,3,0.0,0,Breakfast,Kota Denpasar,Online TA,TA/TO,0,0,0,1,No Deposit,240.0,NaN,0,Personal,0.0,0,2,Check-Out
77447,City Hotel,0,0,2017,December,42,15,0,0,0,0.0,0,Breakfast,Kota Denpasar,Direct,Direct,1,0,1,0,No Deposit,NaN,NaN,0,Personal,0.0,0,0,Check-Out


## Data Preprocessing

### Handle missing values

In [14]:
summary[summary['missing_value'] > 0]

,variable,no_unique,pandas_dtype,missing_value,%_missing_values,unique_value
10,children,5,float64,4,0.003350,"[0.0, 1.0, 2.0, 10.0, 3.0, nan]"
13,city,177,object,488,0.408744,"[Kota Denpasar, Kabupaten Bangka, Kabupaten Sl..."
21,agent,333,float64,16340,13.686238,"[nan, 304.0, 240.0, 303.0, 15.0, 241.0, 8.0, 2..."
22,company,352,float64,112593,94.306893,"[nan, 110.0, 113.0, 270.0, 178.0, 240.0, 154.0..."


* For **'children'**, missing values will be **replaced by 0** because most of the time the adults did not bring their children to stay on the hotel.
* There are 488 missing rows of **'city'** which is around 0.4% of total rows. The missing values can be assumed as **'Unknown'** since there is no prior information about the actual values of these missing values.
* Both **'agent'** and **'company'** have a lot of missing values, 13% and 94% of total rows respectively. Since both of them are unique identifier, the missing values can be **treated as '0'** which means **'No agent'** or **'No company'** that denotes these customer did not use an agent or a company to book the hotel.
* But, due to most of rows are missing values, **'company'** column will be dropped later for the rest of analysis.

In [15]:
df['children'] = df['children'].fillna(0)
    
df['city'] = df['city'].fillna('Unknown')
    
df['agent'] = df['agent'].fillna(0)
    
df['company'] = df['company'].fillna(0)

df.isnull().sum().sum()

0

All missing values already imputed and there is no missing values left.

### Handle duplicate data

In [16]:
df.duplicated().sum()

33261

There are 33261 rows that are duplicated. Before dropping them, let's see the sample of duplicated data. 

In [17]:
df[df.duplicated(keep=False)].reset_index(drop=True).head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_weekdays_nights,adults,children,babies,meal,city,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status
0,Resort Hotel,0,14,2017,September,27,1,0,2,2,0.0,0,Breakfast,Kabupaten Bangka,Online TA,TA/TO,0,0,0,0,No Deposit,240.0,0.0,0,Personal,98.00,0,1,Check-Out
1,Resort Hotel,0,14,2017,September,27,1,0,2,2,0.0,0,Breakfast,Kabupaten Bangka,Online TA,TA/TO,0,0,0,0,No Deposit,240.0,0.0,0,Personal,98.00,0,1,Check-Out
2,Resort Hotel,0,72,2017,September,27,1,2,4,2,0.0,0,Breakfast,Kota Denpasar,Direct,Direct,0,0,0,1,No Deposit,250.0,0.0,0,Personal,84.67,0,1,Check-Out
3,Resort Hotel,0,72,2017,September,27,1,2,4,2,0.0,0,Breakfast,Kota Denpasar,Direct,Direct,0,0,0,1,No Deposit,250.0,0.0,0,Personal,84.67,0,1,Check-Out
4,Resort Hotel,0,70,2017,September,27,2,2,3,2,0.0,0,Dinner,Kabupaten Tangerang,Direct,Direct,0,0,0,0,No Deposit,250.0,0.0,0,Personal,137.00,0,1,Check-Out


Since this dataset do not have an customer unique identifier, there is a possibility this dataset contain a lot of duplicated data. They need to be removed from the dataset in order to reduce the size of the dataset but not removing their useful content.

In [18]:
print(f"Total number of rows before dropping duplicated data: {df.shape}")

df = df.drop_duplicates()

print(f"Total number of rows without duplicated data: {df.shape}")

Total number of rows before dropping duplicated data: (119390, 29)
Total number of rows without duplicated data: (86129, 29)


### Fix incorrect dtypes

In [19]:
summary[summary['pandas_dtype'] == 'float64']

,variable,no_unique,pandas_dtype,missing_value,%_missing_values,unique_value
10,children,5,float64,4,0.003350,"[0.0, 1.0, 2.0, 10.0, 3.0, nan]"
21,agent,333,float64,16340,13.686238,"[nan, 304.0, 240.0, 303.0, 15.0, 241.0, 8.0, 2..."
22,company,352,float64,112593,94.306893,"[nan, 110.0, 113.0, 270.0, 178.0, 240.0, 154.0..."
25,adr,8879,float64,0,0.000000,"[0.0, 75.0, 98.0, 107.0, 103.0, 82.0, 105.5, 1..."


All the variables above  except adr must be in integer because their values are not decimal.

In [20]:
for col in ['children', 'agent', 'company']:
    df[col] = df[col].astype(int)

Now, all variables have correct data types.

### Handle data quality issues

Based on pre-exploration, there are zero total guests and zero total nights in this dataset. Let's exclude this rows from the rest of dataset. 

In [34]:
zero_guest = (
    (df['adults'] == 0) & 
    (df['children'] == 0) & 
    (df['babies'] == 0)
)

zero_night = (
    (df['stays_in_weekend_nights'] == 0) & 
    (df['stays_in_weekdays_nights'] == 0)
)

df = df[~zero_guest].copy()
df_cleaned = df[~zero_night].reset_index(drop=True).copy()
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85378 entries, 0 to 85377
Data columns (total 29 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   hotel                           85378 non-null  object
 1   is_canceled                     85378 non-null  int64 
 2   lead_time                       85378 non-null  int64 
 3   arrival_date_year               85378 non-null  int64 
 4   arrival_date_month              85378 non-null  object
 5   arrival_date_week_number        85378 non-null  int64 
 6   arrival_date_day_of_month       85378 non-null  int64 
 7   stays_in_weekend_nights         85378 non-null  int64 
 8   stays_in_weekdays_nights        85378 non-null  int64 
 9   adults                          85378 non-null  int64 
 10  children                        85378 non-null  int64 
 11  babies                          85378 non-null  int64 
 12  meal                            85378 non-null

### Save the clean dataset

In [45]:
df_cleaned.to_csv("cleaned_hotel_bookings_data.csv", index=False)